# Классы, наследование, агрегация, композиция и примеси 

## Класс 

Класс в Ruby - это объект специального типа. Для его создания достаточно простой конструкции

In [26]:
class ExampleClass
  def initialize(a, b)
  end
end

:initialize

In [4]:
ExampleClass.class.ancestors

[Class, Module, Object, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]

Создание нового экземпляра - через метод `:new`

In [ ]:
ExampleClass.new

ArgumentError: wrong number of arguments (given 0, expected 2)

## Атрибуты класса

Можно объявлять переменные уровня класса (статические) и уровня объекта. Их нужно объявлять в конструкторе. **Конструктор называется `initialize`**.

In [ ]:
class ExampleClass
  def initialize
    @@static = 'static'
    @non_static = 42
  end
end

e = ExampleClass.new
e.@non_static

SyntaxError: <main>:8: syntax error, unexpected instance variable
e.@non_static
  ^~~~~~~~~~~


Так делать при этом нельзя

In [ ]:
ExampleClass.static

SyntaxError: <main>: syntax error, unexpected class variable
ExampleClass.@@static
             ^~~~~~~~


In [14]:
class ExampleClass
  def initialize
    @@static = 'static'
    @non_static = 42
  end
  
  def print
    p @@static
    p @non_static
  end
  
  def set_static(v)
    @@static = v
  end
  
  def set_non_static(v)
    @non_static = v
  end
end

e = ExampleClass.new
e1 = ExampleClass.new

e.print
e1.print

e.set_static('CHANGE')
e.set_non_static('CHANGE')

e.print
e1.print

ExampleClass.new

e.print

"static"
42
"static"
42
"CHANGE"
"CHANGE"
"CHANGE"
42
"static"
"CHANGE"


"CHANGE"

"Статические" переменные разделяют свое значение между всеми экземплярами класса, а каждый экземпляр может это общее значение менять. Такая организация кода приводит к большому количеству ошибок при его усложнении, поэтому **пользоваться `@@` крайне не рекомендую!**

Для того, чтобы переменные уровня объекта были доступны извне, нужно описать соответствующие методы. По соглашению методы называются так же, как и соответствующие им атрибуты (только без `@`):

In [2]:
class ExplicitAttrs
  def initialize(a)
    @a = a
  end
  
  def print
    p @a
  end
  
  def a
    @a
  end
  
  def a=(v)
    @a = v
  end
end

ea = ExplicitAttrs.new 10

ea.print

ea.a = ea.a + 20

ea.print

10
30


30

Если мы хотим, чтобы атрибут был доступен только для чтения или только для записи, мы должны определить один соответствующий метод - или геттер, или сеттер.

Чтобы не мучиться с большим количеством однотипного кода и не загромождать класс, можно пользовать встроенными макросами:

* `attr_reader` создаст только геттер;
* `attr_writer` создаст только сеттер;
* `attr_accessor` можно использовать, если нужен и геттер, и сеттер (то же самое, что написать сначала `attr_reader`, затем `attr_writer`);

In [8]:
class ImplicitAttrs1
  attr_accessor :a, :b, :c
  
  def inspect
    "a: #{a}, b: #{b}, c: #{c}"
  end
end

ia = ImplicitAttrs1.new

a: , b: , c: 

При этом **необязательно объявлять атрибуты в конструкторе (методе `initialize`)**. Начальное значение будет `nil`, а потом 

In [9]:
ia = ImplicitAttrs1.new

a: , b: , c: 

In [10]:
ia.a.nil?

true

In [11]:
ia.a = 1

1

In [12]:
ia.b

In [13]:
ia.b = 10
ia

a: 1, b: 10, c: 

In [14]:
ia.c = 100
ia.c

100

In [15]:
ia

a: 1, b: 10, c: 100

Обычно **нигде, кроме конструктора, не пользуются `@`, а создают `attr_*` в зависимости от того, что нужно**!

## Инкапсуляция

Для инкапсуляции предусмотрены специальные ключевые слова (читай - методы):

In [16]:
class EncapsulationExample
  def public_method
    puts 'public'
  end
  
  protected
  
  def protected1
    puts 'protected1'
  end
  
  def protected2
    puts 'protected2'
  end
  
  private
  
  def private1
    puts 'private1'
  end
  
  public
  
  def also_public
    puts 'also public'
  end
end

:also_public

In [17]:
ee = EncapsulationExample.new

#<EncapsulationExample:0x00007f9ea98cb7f8>

In [18]:
ee.public_method

public


In [19]:
ee.also_public

also public


In [ ]:
ee.protected1

NoMethodError: protected method `protected1' called for #<EncapsulationExample:0x00007f9ea98cb7f8>
Did you mean?  protected2

In [ ]:
ee.private1

NoMethodError: private method `private1' called for #<EncapsulationExample:0x00007f9ea98cb7f8>

Тем не менее, ограничение достаточно легко обходится с помощью встроенного механизма обмена сообщениями (вообще говоря, вызов метода в Ruby - это и есть отправка сообщения).

In [22]:
ee.send(:private1)

private1


## Наследование 

In [27]:
ExampleClass.superclass # Родительский класс.

Object

In [28]:
ExampleClass.ancestors # Цепочка наследования

[ExampleClass, Object, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]

In [29]:
class Figure
  attr_accessor :width, :length
  
  def initialize(width, length)
    @width = width
    @length = length
  end
  
  def square
    raise NotImplementedError, 'Method square is not implemented'
  end
  
  def volume
    raise NotImplementedError, 'Method volume is not implemented'
  end
end

class Rectangle < Figure
  def square
    width * length
  end
end

class Square < Rectangle
  def initialize(s)
    super(s, s)
  end
end

:initialize

In [30]:
r = Rectangle.new(2, 7)
s = Square.new(10)

#<Square:0x00007f9ea38f1df0 @width=10, @length=10>

In [31]:
r.square

14

In [ ]:
r.volume

NotImplementedError: Method volume is not implemented

In [33]:
s.square

100

В Ruby **используется ОДИНОЧНАЯ МОДЕЛЬ НАСЛЕДОВАНИЯ**.

## Агрегация и композиция 

И агрегация, и композиция - это конструирование объекта из нескольких. Иными словами, вы не строите иерархию с кучей методов и ничего не примешиваете, а просто делаете специальный объект для какой-то специальной цели и используете его внутри других объектов.

Классический пример - логирование. Вряд ли имеет хоть какой-то смысл наследоваться от логгера или шарить код логирования через примесь. Помимо концептуальной некорректности абстракции это еще и сложно тестировать и поддерживать.

Посмотрим, как можно реализовать агрегацию и композицию на примере логера.

In [34]:
class MyLogger
  def log(message)
    puts "[#{Time.now.strftime("%Y-%m-%d %H:%M:%S")}] #{message}"
  end
end

:log

In [35]:
MyLogger.new.log('Привет')

[2021-11-11 12:16:55] Привет


Посмотрим, что можно сделать с использованием композиции:

In [36]:
class UserServiceComposition
  
  class << self
    def login
      new.send :login
    end
    
    def logout
      new.send :logout
    end
  end
  
  private
  
  attr_reader :logger
  
  def initialize
    @logger = MyLogger.new
  end
  
  def login
    logger.log('User login')
  end
  
  def logout
    logger.log('User logout')
  end
  
end

:logout

In [37]:
UserServiceComposition.login

[2021-11-11 12:22:06] User login


In [38]:
UserServiceComposition.logout

[2021-11-11 12:22:07] User logout


Класс **сам управляет своими зависимостями**.

То же самое с использованием агрегации:

In [64]:
class UserServiceAggregation
  def initialize(logger)
    @logger = logger
  end
  
  def login
    logger.log('User login')
  end
  
  def logout
    logger.log('User logout')
  end
  
  private
  
  attr_reader :logger
end

class DependencyInjector
  @instance = new

  private_class_method :new

  def initialize
    @dependecies = Hash.new
    @dependecies[MyLogger] = MyLogger.new
    @dependecies[UserServiceAggregation] = UserServiceAggregation.new(@dependecies[MyLogger])
  end
  
  attr_reader :dependecies
  
  public
  
  class << self    
    def get(class_name)
      instance.dependecies[class_name]
    end
  end
end

:get

In [65]:
service = DependencyInjector.get(UserServiceAggregation)

#<UserServiceAggregation:0x00007fc8828b8e20 @logger=#<MyLogger:0x00007fc8828b8e48>>

In [66]:
service.login

[2020-10-28 11:09:19] User login


Зависимостями **управляет кто-то извне**.

В конечном счете агрегация удобнее, потому что позволяет легко подменять реальные классы (это ведь может быть не только логер, но и сервис для работы с БД, например) на какие-то другие.

Если вы соблюдаете интерфейс, вы можете:

* перейти с одного источника данных на другой;
* сделать "мок" - более простую реализацию, например, для окружения разработчика или пока ждете реализации сервисов со стороны коллег;
* легко тестировать.

## Примеси

Наследование как концепт хорошо в том случае, когда наследник - это **более специфичная версия родителя**, иногда оно **не нужно**.

Одним из способов получить чистую архитектуру является [выбор в пользу композиции перед наследованием](https://en.wikipedia.org/wiki/Composition_over_inheritance). Иными словами вы:

1. Разбиваете поведение большого и сложного класса по нескольким простым, каждый из которых выполняет какую-то одну функцию.
2. Не наследуетесь от этих классов, а собираете их внутри своего класса.

Представим себе, что мы создаем библиотеку песен. Мы хотим, чтобы у нас были классы, отвечающие за сборник, песню, исполнителя. Помимо этого, мы хотим сделать так, чтобы можно было сравнивать песни между собой, сортировать их, а также чтобы у нас была возможность отладки. 

In [39]:
module Debug
  def who_am_i
    "#{self.class.name} (\##{object_id}): #{to_s}"
  end
end

module IsArray
  def as_array value
    value.kind_of?(Array) ? value : [value]
  end
end

class Song
  include Comparable, Debug, IsArray
  
  attr_accessor :title, :artists, :duration_sec, :text
  
  def initialize(title, artists, duration_sec, text=nil)
    @title = title
    @artists = as_array(artists)
    @duration_sec = duration_sec
    @text = text
  end
  
  def artists=(v)
    @artists = as_array(v)
  end
  
  def inspect
    "#{title} (#{artists.join(', ')} - #{duration_sec} sec.)"
  end
  
  def <=>(other)
    duration_sec <=> other.duration_sec
  end
end

class Album
  include Debug, IsArray
  
  attr_accessor :title, :artists, :year, :songs
  
  def initialize(title, artists, year, songs=[])
    @title = title
    @artists = as_array(artists)
    @year = year
    @songs = as_array(songs)
  end
  
  def artists=(v)
    @artists = as_array(v)
  end
  
  def songs=(v)
    @songs = as_array(v)
  end
  
  def print
    puts "ALBUM #{title}"
    puts "BY #{artists.join(', ')}"
    puts year.to_s
    puts songs
      .map(&:inspect)
      .each_with_index
      .map { |name, idx| "#{idx + 1}. #{name}" }
      .join("\n")
  end
  
  def sorted_songs
    songs.sort
  end
  
  class << self
    def from_song_list(title, artists, year, songs)
      album = new title, artists, year
      album.songs = songs.map do |song_info|
        Song.new(
          song_info[:title],
          song_info[:artists] || album.artists,
          song_info[:duration_sec],
          song_info[:text]
        )
      end
      album
    end
  end
end

:from_song_list

In [40]:
album = Album.from_song_list(
  'Diamonds Deluxe',
  ['Elthon John'],
  2017,
  [
    {
      title: 'Your Song',
      duration_sec: 243

    },
    {
      title: "Rocket Man (I Think It's Going To Be A Long Long Time)",
      duration_sec: 282
    },
    {
      title: 'Goodbye Yellow Brick Road',
      duration_sec: 192,
      text: <<-eos
When are you gonna come down?
When are you going to land?
I should have stayed on the farm
I should’ve listened to my old man
You cannot hold me forever
I didn’t sign up with you
I’m not a present for your friends to open
This girl’s to young to be singing
The blues
So goodbye yellow brick road
Where the dogs of society howl
You can’t plant me in your penthouse
I’m going back to my plough
Back to the howlin' old owl in the woods
Hunting the horny black toad
Oh I’ve finally decided my future lies
Beyond the yellow brick road
What do you think you’ll do then?
They’ll probably shoot down your plane
It’ll take you a couple vodka and tonics
To set you on your feet again
Maybe you’ll get a replacement
There’s plenty like me to be found
Mongrels who ain’t got a penny
Sniffing for tidbits like you
So goodbye yellow brick road
Where the dogs of society howl
You can’t plant me in your penthouse
I’m going back to my plough
Back to the howlin' old owl in the woods
Hunting the horny black toad
Oh I’ve finally decided my future lies
Beyond the yellow brick road
eos
    },
    {
      title: "Don't Go Breaking My Heart",
      artists: ['Elton John', 'Kiki Dee'],
      duration_sec: 266
    }
  ]
)

album.print

ALBUM Diamonds Deluxe
BY Elthon John
2017
1. Your Song (Elthon John - 243 sec.)
2. Rocket Man (I Think It's Going To Be A Long Long Time) (Elthon John - 282 sec.)
3. Goodbye Yellow Brick Road (Elthon John - 192 sec.)
4. Don't Go Breaking My Heart (Elton John, Kiki Dee - 266 sec.)


In [41]:
album.sorted_songs

[Goodbye Yellow Brick Road (Elthon John - 192 sec.), Your Song (Elthon John - 243 sec.), Don't Go Breaking My Heart (Elton John, Kiki Dee - 266 sec.), Rocket Man (I Think It's Going To Be A Long Long Time) (Elthon John - 282 sec.)]

In [43]:
album.songs.first.who_am_i

"Song (#1320): #<Song:0x00007f9ea60e9ce0>"

In [45]:
album.artists = ['Elthon John']
album.artists

["Elthon John"]

Примеси - это способ **горизонтально распространять код** и их нужно предпочитать наследованию когда это возможно.

Чем еще можно заменить наследование?

## Enumerable, Enumerator 

`Enumerable` - это примесь, в которой реализована куча полезных методов по работе с коллекциями. Единственное, что нужно примеси - это метод для итерированию по коллекции, по умолчанию - `each`. Самое смешное, что объект, к которому мы примешиваем `Enumerable`, строго говоря не должен быть именно коллекцией (массивом, например).

In [46]:
class MyCollection
  include Enumerable
  
  attr_reader :name, :surname
  
  def initialize
    @name = 'Иван'
    @surname = 'Иванов'
  end
  
  def each
    yield name
    yield surname
  end
end

:each

In [47]:
MyCollection.new.map { |x| x }

["Иван", "Иванов"]

In [50]:
MyCollection.new.filter { |x| x == 'Иван' }

["Иван"]

`Enumerator` - это то же самое, что и `Enumerable`, только его можно создавать как объект (потому что это и есть объект).

In [52]:
e = Enumerator.new do |yielder|
  yielder.yield 'Иван'
  yielder.yield 'Иванов'
end

e.take(1)

["Иван"]

Пропробуем решить задачу

Найти первый член последовательности `y = n / (n**2 + 2)`, для которого y < ξ. Определить, как изменяется число итераций при изменении
точности.

In [53]:
eps = 0.001

enum = Enumerator.new do |yielder|
  n = 1
  
  loop do
    yielder << n / (n**2 + 2).to_f
    n += 1
  end
end

#<Enumerator: #<Enumerator::Generator:0x00007f9ea98ab250>:each>

In [54]:
enum.find { |y| y < eps }

0.000999998000004

## Ленивые вычисления 

Классический пример.

Просто 5 чисел:

In [55]:
(1..Float::INFINITY).take(5)

[1, 2, 3, 4, 5]

Пять четных чисел:

In [ ]:
(1..Float::INFINITY).filter(&:even?).take(5)

Interrupt: 

Пять ленивых четных чисел

In [57]:
(1..Float::INFINITY).lazy.filter(&:even?).take(5).to_a

[2, 4, 6, 8, 10]

Как это работает

In [58]:
(1..10).map do |x|
  puts "Map 1 #{x}"
  x + 1
end.map do |x|
  puts "Map 2 #{x}"
  x * 2
end

Map 1 1
Map 1 2
Map 1 3
Map 1 4
Map 1 5
Map 1 6
Map 1 7
Map 1 8
Map 1 9
Map 1 10
Map 2 2
Map 2 3
Map 2 4
Map 2 5
Map 2 6
Map 2 7
Map 2 8
Map 2 9
Map 2 10
Map 2 11


[4, 6, 8, 10, 12, 14, 16, 18, 20, 22]

In [59]:
(1..10).lazy.map do |x|
  puts "Map 1 #{x}"
  x + 1
end.map do |x|
  puts "Map 2 #{x}"
  x * 2
end.force

Map 1 1
Map 2 2
Map 1 2
Map 2 3
Map 1 3
Map 2 4
Map 1 4
Map 2 5
Map 1 5
Map 2 6
Map 1 6
Map 2 7
Map 1 7
Map 2 8
Map 1 8
Map 2 9
Map 1 9
Map 2 10
Map 1 10
Map 2 11


[4, 6, 8, 10, 12, 14, 16, 18, 20, 22]

Иными словами, `lazy` прокидывает каждое значение сразу через все блоки, а обычный энумератор - "накапливает" у начала каждого блока.

## Proc, блок и lambda

В третьей часть шестой лабораторной работы вам необходимо сделать так, чтобы некий кусок кода (в зависимости от задания) мог быть передан и как блок, и как Proc/lambda.

Попробуем решить такую задачу

Составить метод scale отыскания масштаба графического (выведенного в текстовом режиме) изображения функции f(x) на экране размером B единиц растра по формуле `M = B / max(f(x))`. В основной программе использовать метод для отыскания масштаба функций `x · sin(x)` и `tg(x)`, при `−2 < x < 2`. Реализовать вызов метода двумя способами: в виде передаваемого lambda-выражения и в виде блока.

Теперь эталонное решение будет таким

In [107]:
def find_scale(left, right, b)
  max = (left..right).step(0.01).map { |x| yield x }.max
  b / max if max != 0 || nil
end

:find_scale

Вызов с использованием блока

In [108]:
find_scale(-2, 2, 10) { |x| Math.sin(x) * x }

5.4987508514730825

Вызов с использованием `proc`

In [109]:
find_scale(-2, 2, 10, &proc { |x| Math.sin(x) * x })

5.4987508514730825

Все это работает через один и тот же `yield`, настолько быстро, насколько это возможно, и максимально идиоматично.